In [101]:
import pandas as pd
import numpy as np   
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import RandomizedSearchCV
import pickle  
from sklearn.ensemble import GradientBoostingRegressor 
from pprint import pprint 

## Reading Data

In [102]:
data = pd.read_csv('../data/clean_data.csv')

In [103]:
data.head()

,السعر,الموديل,موديل سنة,لون السيارة,قوة الماتور,عدد الركاب,عداد السيارة,أصحاب سابقون,أصل السيارة,رخصة السيارة,...,نوع الجير,الزجاج,وسادة حماية هوائية,فرش جلد,جنطات مغنيسيوم,فتحة سقف,مسجل CD,إغلاق مركزي,مُكيّف,جهاز إنذار
0,100000.0,كيا اوبتيما,2014,أبيض عاجي,2000.0,5.0,75000.0,1.0,خصوصي,فلسطينية,...,اوتوماتيك,الكتروني,1,1,1,1,1,1,1,1
1,60000.0,كيا سورينتو,2007,سكني,2500.0,8.0,130000.0,2.0,خصوصي,فلسطينية,...,نصف اوتوماتيك,الكتروني,1,1,1,1,1,1,1,1
2,43500.0,هونداي افانتي,2006,سكني,1600.0,5.0,65000.0,2.0,خصوصي,فلسطينية,...,اوتوماتيك,الكتروني,1,1,1,0,1,1,1,1
3,5500.0,فيات 127,1982,بيج,906.0,5.0,65000.0,0.0,خصوصي,فلسطينية,...,عادي,يدوي,0,0,0,0,1,0,1,0
4,54000.0,بيجو 208,2014,فضي,1200.0,5.0,38000.0,2.0,خصوصي,فلسطينية,...,اوتوماتيك,الكتروني,1,0,1,0,0,1,1,1


#### Data cleaning was done in the previous version, this dataset was exported after the cleaning process

## Feature Enginnering

In [104]:
#get categorical features name
categorical_features = [col for col in data.columns if data[col].dtypes == 'O']

In [105]:
ohe = OneHotEncoder(sparse = False, handle_unknown = 'ignore') 

In [106]:
#define transformer to selectively apply data preparation transforms
transformer = ColumnTransformer( 
    transformers = [  
        #apply onehot encoding to categorical features
        ('categorical', ohe, categorical_features),  
    ],
    #unspecified columns will be passed through without transformation  
    remainder='passthrough',
    #prevent adding prefix to columns names
    verbose_feature_names_out = False 
)

In [107]:
#slice the target variable and the other features
X = data.drop(columns = 'السعر')
y = data['السعر'] 
#transform the data (without the target variable) 
transformed_X = transformer.fit_transform(X)

## Hyperparamete optimization for `Gradient Boosting Tree`

In [109]:
gbt = GradientBoostingRegressor() 

In [120]:
#check for gbt parameters and thier default values
pprint(gbt.get_params())

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}


Define the grid for gradient boosting trees

In [108]:
#Number of features to consider
max_features = ['sqrt', 'log2', None] 
#Number of trees 
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)]  
#Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 74, num = 10)]  
max_depth.append(None)
#Minimum number of samples required to split a node
min_samples_split = [int(x) for x in np.linspace(100, 300, num = 10)]
learning_rate = [0.01, 0.05, 0.1] 
#Create the random grid
random_grid = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split, 
    'learning_rate': learning_rate,
    'max_features': max_features
    } 

In [110]:
#define gbt_random a RandomizedSearchCV obj to search across 30 random different combinations 
#for best hyperparameters for gradient boosting tree regressor using 5 fold cross validation 
gbt_random = RandomizedSearchCV(estimator = gbt, 
                               param_distributions = random_grid,  
                               n_iter = 30,  
                               cv = 5, 
                               verbose=4,  
                               n_jobs = 4) 

In [111]:
#Fit the random search model  
gbt_random.fit(transformed_X, y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_iter=30,
                   n_jobs=4,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1],
                                        'max_depth': [5, 12, 20, 28, 35, 43, 51,
                                                      58, 66, 74, None],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_split': [100, 122, 144,
                                                              166, 188, 211,
                                                              233, 255, 277,
                                                              300],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   verbose=4)

In [112]:
#get the best estimator
best_gbt_model = gbt_random.best_estimator_

In [113]:
print(f"Best parameters:\n {gbt_random.best_params_}\n\n Best score\n {gbt_random.best_score_}")

Best parameters:
 {'n_estimators': 300, 'min_samples_split': 211, 'max_features': 'sqrt', 'max_depth': 74, 'learning_rate': 0.05}

 Best score
 0.7935723340350735


# Pickling

In [114]:
#GBTRegressor Pipeline
gbt_pipeline = Pipeline( 
    steps = [
        ("transformer", transformer),
        ("gradient boosting tree regressor", best_gbt_model)
    ] 
)
#fit data
gbt_pipeline.fit(X, y) 

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['الموديل', 'لون السيارة',
                                                   'أصل السيارة',
                                                   'رخصة السيارة', 'نوع الوقود',
                                                   'نوع الجير', 'الزجاج'])],
                                   verbose_feature_names_out=False)),
                ('gradient boosting tree regressor',
                 GradientBoostingRegressor(learning_rate=0.05, max_depth=74,
                                           max_features='sqrt',
                                           min_samples_split=211,
                                           n_estimators=300))])

In [115]:
#export gbt regressor pipeline 
gbt_regressor_pipline_file_name = "../models/gbt pipline.pkl"
pickle.dump(gbt_pipeline, open(gbt_regressor_pipline_file_name, 'wb'))